In [1]:
%matplotlib inline


# 3D scattering quantum chemistry regression

Description:
This example trains a classifier combined with a scattering transform to
regress molecular atomization energies on the QM7 dataset. Here, we use full
charges, valence charges and core charges. A linear regression is deployed.

Remarks:
The linear regression of the QM7 energies with the given values gives MAE
2.75, RMSE 4.18 (kcal.mol-1)

Reference:
https://arxiv.org/abs/1805.00571


## Preliminaries

First, we import NumPy, PyTorch, and some utility modules.



In [2]:
import numpy as np
import torch
import time
import os

We will use scikit-learn to construct a linear model, so we import the
necessary modules. In addition, we need to compute distance matrices when
normalizing our input features, so we import `pdist` from `scipy.spatial`.



In [3]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

We then import the necessary functionality from Kymatio. First, we need the
PyTorch frontend of the 3D solid harmonic cattering transform.



In [4]:
from kymatio.torch import HarmonicScattering3D

The 3D transform doesn't compute the zeroth-order coefficients, so we need
to import `compute_integrals` to do this manually.



In [5]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

To generate the input 3D maps, we need to calculate sums of Gaussians, so we
import the function `generate_weighted_sum_of_gaussians`.



In [6]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

Finally, we import the utility functions that let us access the QM7 dataset
and the cache directories to store our results.



In [7]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

## Data preparation

Fetch the QM7 database and extract the atomic positions and nuclear charges
of each molecule. This dataset contains 7165 organic molecules with up to
seven non-hydrogen atoms, whose energies were computed using density
functional theory.



In [8]:
import sys
sys.path.append(os.path.abspath(".."))
from utils_project import generate_csv,create_dataframe_from_xyz_files,create_X_y_from_dataframe


csv_path = "../../data/energies/train.csv"
path_data = "../../data/atoms/train"
df_train=create_dataframe_from_xyz_files(path_data,csv_path)
X=df_train[['positions', 'energy', 'charges']]

qm7 = X.to_dict("list")

#qm7 = fetch_qm7(align=True)
pos = np.array(qm7['positions'])
full_charges = np.array(qm7['charges'])

n_molecules = pos.shape[0]

In [9]:
for clé, valeur in qm7.items():
    print(f"Clé: {clé}, Haut de la liste: {valeur[:30]}")

print(qm7.keys())

Clé: positions, Haut de la liste: [array([[-2.452236, -0.446061, -0.162506],
       [-1.075187, -0.275346,  0.122104],
       [-0.369125,  0.521836, -0.694705],
       [ 0.971233,  0.690216, -0.567978],
       [ 1.760863,  0.025606,  0.422999],
       [ 2.433176, -0.494864,  1.207176],
       [-2.940898, -0.820652,  0.740555],
       [-2.59426 , -1.178418, -0.972529],
       [-2.917918,  0.503649, -0.463334],
       [-0.935551,  1.052651, -1.474088],
       [ 1.49348 ,  1.359999, -1.244527],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ]]

From the nuclear charges, we compute the number of valence electrons, which
we store as the valence charge of that atom.



In [10]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

We then normalize the positions of the atoms. Specifically, the positions
are rescaled such that two Gaussians of width `sigma` placed at those
positions overlap with amplitude less than `overlapping_precision`.



In [11]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

## Scattering Transform
Given the rescaled positions and charges, we are now ready to compute the
density maps by placing Gaussians at the different positions weighted by the
appropriate charge. These are fed into the 3D solid harmonic scattering
transform to obtain features that are used to regress the energies. In
order to do this, we must first define a grid.



In [12]:
M, N, O = 64, 64, 64 #192, 128, 96
grille = "64-64-64"
grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

We then define the scattering transform using the `HarmonicScattering3D`
class.



In [13]:
J = 2
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

We then check whether a GPU is available, in which case we transfer our
scattering object there.



In [14]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)
print(device)

cuda


The maps computed for each molecule are quite large, so the computation has
to be done by batches. Here we select a small batch size to ensure that we
have enough memory when running on the GPU. Dividing the number of molecules
by the batch size then gives us the number of batches.



In [15]:
batch_size = 8
n_batches = int(np.ceil(n_molecules / batch_size))

We are now ready to compute the scattering transforms. In the following
loop, each batch of molecules is transformed into three maps using Gaussians
centered at the atomic positions, one for the nuclear charges, one for the
valence charges, and one with their difference (called the “core” charges).
For each map, we compute its scattering transform up to order two and store
the results.



In [16]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules, "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None

for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
            i + 1, int(((n_batches - i - 1) * dt) // 3600),
            int((((n_batches - i - 1) * dt) // 60) % 60),
            int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # === Nuclear density ===
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
                                pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch).float().to(device)

    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch, integral_powers)
    full_scattering = scattering(full_density_batch)

    # === Valence density ===
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
                                pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch).float().to(device)

    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch, integral_powers)
    val_scattering = scattering(val_density_batch)

    # === Core density ===
    core_density_batch = full_density_batch - val_density_batch
    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch, integral_powers)
    core_scattering = scattering(core_density_batch)

    # === Stack all coefficients ===
    batch_order_0 = torch.stack((full_order_0, val_order_0, core_order_0), dim=-1).to(device)
    batch_orders_1_and_2 = torch.stack((full_scattering, val_scattering, core_scattering), dim=-1).to(device)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)


Computing solid harmonic scattering coefficients of 6591 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:41:05]
Iteration 3 ETA: [00:34:54]
Iteration 4 ETA: [00:36:11]
Iteration 5 ETA: [00:34:48]
Iteration 6 ETA: [00:34:43]
Iteration 7 ETA: [00:35:13]
Iteration 8 ETA: [00:36:08]
Iteration 9 ETA: [00:35:25]
Iteration 10 ETA: [00:38:22]
Iteration 11 ETA: [00:37:11]
Iteration 12 ETA: [00:35:42]
Iteration 13 ETA: [00:37:50]
Iteration 14 ETA: [00:36:55]
Iteration 15 ETA: [00:36:23]
Iteration 16 ETA: [00:36:28]
Iteration 17 ETA: [00:37:50]
Iteration 18 ETA: [00:36:38]
Iteration 19 ETA: [00:36:31]
Iteration 20 ETA: [00:37:55]
Iteration 21 ETA: [00:37:24]
Iteration 22 ETA: [00:37:21]
Iteration 23 ETA: [00:36:54]
Iteration 24 ETA: [00:36:42]
Iteration 25 ETA: [00:36:57]
Iteration 26 ETA: [00:36:18]
Iteration 27 ETA: [00:35:05]
Iteration 28 ETA: [00:36:49]
Iteration 29 ETA: [00:37:21]
Iteration 30 ETA: [

In [17]:
# Fusionner les résultats en un seul tenseur
order_0_tensor = torch.cat(order_0, dim=0)
orders_1_and_2_tensor = torch.cat(orders_1_and_2, dim=0)

# Sauvegarder les tenseurs sur disque
torch.save({
    'order_0': order_0_tensor,
    'orders_1_and_2': orders_1_and_2_tensor
}, f'../models_scattering/scattering_outputs_{grille}.pt')

In [18]:
# Charger les données
saved_data = torch.load( f'../models_scattering/scattering_outputs_{grille}.pt', map_location=device)
order_0 = saved_data['order_0']
orders_1_and_2 = saved_data['orders_1_and_2']

Concatenate the batch outputs and transfer to NumPy.



In [20]:
order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

## Regression

To use the scattering coefficients as features in a scikit-learn pipeline,
these must be of shape `(n_samples, n_features)`, so we reshape our arrays
accordingly.



In [21]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

Since the above calculation is quite lengthy, we save the results to a cache
for future use.



In [22]:
basename = 'qm7_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("qm7/experiments")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2)

We now concatenate the zeroth-order coefficients with the rest since we want
to use all of them as features.



In [23]:
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)

Fetch the target energies from the QM7 dataset.



In [24]:
target = qm7['energy']

We evaluate the performance of the regression using five-fold
cross-validation. To do so, we first shuffle the molecules, then we store
the resulting indices in `cross_val_folds`.



In [25]:
n_folds = 3

P = np.random.permutation(n_molecules).reshape((n_folds, -1))

cross_val_folds = []

for i_fold in range(n_folds):
    fold = (np.concatenate(P[np.arange(n_folds) != i_fold], axis=0),
            P[i_fold])
    cross_val_folds.append(fold)

Given these folds, we compute the regression error for various settings of
the `alpha` parameter, which controls the amount of regularization applied
to the regression problem (here in the form of a simple ridge regression, or
Tikhonov, regularization). The mean absolute error (MAE) and root mean
square error (RMSE) is output for each value of `alpha`.



In [26]:
import numpy as np
from sklearn import linear_model, preprocessing, pipeline, model_selection

# Supposons que scattering_coef et target soient déjà définis
alphas = 10.0 ** (-np.arange(1, 10))
results = []

for alpha in alphas:
    scaler = preprocessing.StandardScaler()
    ridge = linear_model.Ridge(alpha=alpha)
    regressor = pipeline.make_pipeline(scaler, ridge)

    target_prediction = model_selection.cross_val_predict(regressor, X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    results.append((alpha, MAE, RMSE))

    print('Ridge regression, alpha: {}, MAE: {}, RMSE: {}'.format(alpha, MAE, RMSE))

# Trouver le modèle avec le RMSE le plus bas
best_result = min(results, key=lambda x: x[2])
best_alpha, best_mae, best_rmse = best_result

print(f"Le meilleur modèle a un alpha de {best_alpha} avec un RMSE de {best_rmse}.")


/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=2.45162e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=2.39053e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=2.80978e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge regression, alpha: 0.1, MAE: 0.2874292775695367, RMSE: 0.7081470857341379
Ridge regression, alpha: 0.01, MAE: 0.26608618707456827, RMSE: 1.3107511846621902
Ridge regression, alpha: 0.001, MAE: 0.27304462101326055, RMSE: 3.120425339947383
Ridge regression, alpha: 0.0001, MAE: 0.32735909157175985, RMSE: 6.459791904528879
Ridge regression, alpha: 1e-05, MAE: 0.4028115877029406, RMSE: 10.691861199003817
Ridge regression, alpha: 1e-06, MAE: 0.4455677348250949, RMSE: 12.875741186675501
Ridge regression, alpha: 1e-07, MAE: 0.4807934821404321, RMSE: 14.329061362477534
Ridge regression, alpha: 1e-08, MAE: 0.4884990573278494, RMSE: 14.616689892438746
Ridge regression, alpha: 1e-09, MAE: 0.46937412049421595, RMSE: 13.672167757115337
Le meilleur modèle a un alpha de 0.1 avec un RMSE de 0.7081470857341379.


In [27]:
import numpy as np
from sklearn import linear_model, preprocessing, pipeline, model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import joblib

# Supposons que scattering_coef et target soient déjà définis
cross_val_folds = 5  # Assurez-vous que cross_val_folds est défini

# Liste des modèles à tester
models = [
    ("Ridge Regression with alpha=0.1", linear_model.Ridge(alpha=0.1)),
    ("Ridge Regression with alpha=1", linear_model.Ridge(alpha=1)),
    ("Ridge Regression with alpha=10", linear_model.Ridge(alpha=10)),
    ("Lasso Regression", linear_model.Lasso()),
    ("ElasticNet Regression", linear_model.ElasticNet()),
    ("Random Forest Regression", RandomForestRegressor()),
    ("Support Vector Regression", SVR()),
    ("XGBoost Regression", XGBRegressor()),
    ("MLP Regressor", MLPRegressor(random_state=1, max_iter=2000, tol=0.1))
]

results = []

for name, model in models:
    scaler = preprocessing.StandardScaler()
    regressor = pipeline.make_pipeline(scaler, model)

    target_prediction = model_selection.cross_val_predict(regressor, X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    results.append((name, model, MAE, RMSE))

    print('{}: MAE: {}, RMSE: {}'.format(name, MAE, RMSE))

# Trouver le modèle avec le RMSE le plus bas
best_result = min(results, key=lambda x: x[3])
best_model_name, best_model, best_mae, best_rmse = best_result

print(f"Le meilleur modèle est {best_model_name} avec un RMSE de {best_rmse}.")

/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.99365e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.96524e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.97138e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.77525e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/tristan/.local/lib/python3.10/site-package

Ridge Regression with alpha=0.1: MAE: 0.2799451085647929, RMSE: 0.6534790099081015
Ridge Regression with alpha=1: MAE: 0.3484688564678764, RMSE: 0.680133780836642
Ridge Regression with alpha=10: MAE: 0.48519955388342456, RMSE: 0.8574156743790448
Lasso Regression: MAE: 2.3079641142090592, RMSE: 3.04812295712579


/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.991e+01, tolerance: 7.168e+01
  model = cd_fast.enet_coordinate_descent(


ElasticNet Regression: MAE: 2.175899556311856, RMSE: 2.857220461389506
Random Forest Regression: MAE: 0.3310212526653286, RMSE: 1.146138283878792
Support Vector Regression: MAE: 1.1179461673320066, RMSE: 2.3121919724390443
XGBoost Regression: MAE: 0.3811350695404571, RMSE: 1.168573582753926
MLP Regressor: MAE: 1.9689463726840528, RMSE: 8.203589725310144
Le meilleur modèle est Ridge Regression with alpha=0.1 avec un RMSE de 0.6534790099081015.


In [28]:
# Entraîner le meilleur modèle sur l'ensemble des données
scaler = preprocessing.StandardScaler()
best_regressor = pipeline.make_pipeline(scaler, best_model)
best_regressor.fit(scattering_coef, target)

# Enregistrer le meilleur modèle
joblib.dump(best_regressor, f'../models_scattering/best_model_{grille}.pkl')

/home/tristan/.local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.50622e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


['../models_scattering/best_model_64-64-64.pkl']